# EDS-TEM quantification of core shell nanoparticles

* Using machine learning methods, the composition of embedded <br/>nanostructures can be accurately measured 
* Demonstrated by D. Roussow et al., Nano Letters, 2015 
    * See the [full article](https://www.repository.cam.ac.uk/bitstream/handle/1810/248102/Roussouw%20et%20al%202015%20Nano%20Letters.pdf?sequence=1) for details 
* Using the same data, this notebook reproduces the main results of this article

In [ ]:
%matplotlib nbagg
import hyperspy.api as hs
import numpy as np

In [ ]:
import logging
hs_logger = logging.getLogger('hyperspy') 
hs_logger.setLevel(logging.ERROR)

## Credits

* This notebook was originally written by Pierre Burdet in 2015, with subsequent edits by:
    * Duncan Johnstone &mdash; 2016
    * Francisco de la Peña &mdash; 2016
    * Pierre Burdet &mdash; 2016
    * Andy Herzing and Josh Taillon &mdash; 2018
    

* Requires HyperSpy v1.3+

## <a id='top'></a> Contents

1. <a href='#dat'> Specimen & Data</a>
2. <a href='#loa'> Loading</a>
3. <a href='#bss'> Blind source separation of core/shell nanoparticles</a>
4. <a href='#bare'> Representative spectrum from bare cores</a>
5. <a href='#com'> Comparison and quantification</a>
6. <a href='#fur'> Going father: Isolating the nanoparticles</a>

# <a id='dat'></a> 1. Specimen & Data

The sample and the data used in this tutorial are described in 
D. Roussow, et al., Nano Letters, In Press (2015) (see the [full article](https://www.repository.cam.ac.uk/bitstream/handle/1810/248102/Roussouw%20et%20al%202015%20Nano%20Letters.pdf?sequence=1)).

FePt@Fe$_\mathsf{3}$O$_\mathsf{4}$ core-shell nanoparticles are investigated with an EDS/TEM experiment (FEI Osiris TEM, 4 EDS detectors). The composition of the core can be measured with ICA (see figure 1c). To prove the accuracy of the results, measurements on bare FePt bimetallic nanoparticles from a synthesis prior to the shell addition step are used.

<center><img src="img/core_shell.png" style="height:550px;"></center>

###### **Figure 1:** (a) A spectrum image obtained from a cluster of core-shell nanoparticles. (b) The nanoparticles are comprised of a bi-metallic Pt/Fe core surrounded by an iron oxide shell on a carbon support. (c) ICA decomposes the mixed EDX signals into components representing the core (IC#0), shell (IC#1) and support (IC#2).

In [ ]:
# extract data locally since we do not have access to Dropbox
# from urllib.request import urlretrieve, urlopen
from zipfile import ZipFile
# files = urlretrieve("https://www.dropbox.com/s/ecdlgwxjq04m5mx/HyperSpy_demos_EDS_TEM_files.zip?raw=1", "./HyperSpy_demos_EDX_TEM_files.zip")
with ZipFile("examples/TEM_EDS_nanoparticles_files.zip") as z:
    z.extractall(path='examples/')

# <a id='loa'></a> 2. Loading the data

<a href='#top'> Table of contents</a>

Import HyperSpy, numpy and matplotlib libraries

In [ ]:
%matplotlib nbagg
import hyperspy.api as hs
import numpy as np

Load the spectrum images of the bare seeds and the core shell nanoparticles.

In [ ]:
c = hs.load('examples/bare_core.hdf5')
cs = hs.load('examples/core_shell.hdf5')

In [ ]:
# Examine the metadata of the cores:
c.metadata

Plot the intensity of Fe K${\alpha}$ and Pt L${\alpha}$ lines:

In [ ]:
axes = hs.plot.plot_images(hs.transpose(*(c.get_lines_intensity() + cs.get_lines_intensity())),
                           scalebar='all', axes_decor='off', per_row=2, cmap='viridis')

## <a id='bss'></a> 3. Blind source separation of core/shell nanoparticles

<a href='#top'> Table of contents</a>

Apply blind source separation (ICA) to obtain a factor (spectrum) corresponding to the core.

In [ ]:
# Have to change datatype to float for decomposition:
cs.change_dtype('float')
cs.decomposition()

In [ ]:
ax = cs.plot_explained_variance_ratio()

ICA on the three first components.

In [ ]:
cs.blind_source_separation(3)

In [ ]:
axes = cs.plot_bss_loadings()

In [ ]:
axes = cs.plot_bss_factors()

The first component corresponds to the core.

In [ ]:
s_bss = cs.get_bss_factors().inav[0]

In [ ]:
s_bss.plot()

## <a id='bare'></a> 4. Representative spectrum from bare cores

<a href='#top'> Table of contents</a>

We meed to obtain a representative spectrum of the bare nanoparticles
so we can compare to the BSS component

We can mask the low intensity of the Pt L${\alpha}$ signal:

In [ ]:
pt_la = c.get_lines_intensity(['Pt_La'])[0]
mask = pt_la > 6

Visualizing the mask:

In [ ]:
axes = hs.plot.plot_images(hs.transpose(*(mask, pt_la * mask)), axes_decor='off', colorbar=None,
                           label=['Mask', 'Pt L${\\alpha}$ intensity'], cmap='viridis')

Applying the mask:

* `mask` is a `Signal` containing boolean values, but it is 2D, not 3D:

In [ ]:
print(mask.data.shape)
mask.data

* To apply the mask, we can just multiply the signals together thanks to `numpy`'s array broadcasting:

In [ ]:
c_masked = c * mask

In [ ]:
c_masked.plot()

The sum over the masked particles is used as a bare core spectrum:

In [ ]:
s_bare = c_masked.sum()

In [ ]:
s_bare.plot()

## <a id='com'></a> 5. Comparison and quantification

<a href='#top'> Table of contents</a>

We stack together the spectrum of bare particles and the first ICA component:

In [ ]:
s_bare.change_dtype('float')
s = hs.stack([s_bare, s_bss], new_axis_name='Bare or BSS')
s.metadata.General.title = 'Bare or BSS'

In [ ]:
# Normalize the data for plotting for easier comparison:
axes = hs.plot.plot_spectra([sig/sig.data.max() for sig in s], style='cascade', 
                            legend=['Bare particles', 'BSS #0'])

### Comparison method 1 &mdash; net intensity calculation

X-ray intensities measurement with background subtraction

In [ ]:
w = s.estimate_background_windows()

In [ ]:
s.plot(background_windows=w, navigator='slider')

Refinement of the windows position.

In [ ]:
w

In [ ]:
w[1, 0] = 8.44
w[1, 1] = 8.65

In [ ]:
s.plot(background_windows=w, navigator='slider')

Get the net intensity under the peaks (using our new background windows):

In [ ]:
sI = s.get_lines_intensity(background_windows=w)
sI

Comparing the ratio of Fe intensity to Pt:

In [ ]:
print('Bare core Fe_Ka/Pt_La: \t{:.2f}'.format(sI[0].data[0] / sI[1].data[0]))
print('BSS Fe_Ka/Pt_La: \t{:.2f}'.format(sI[0].data[1] / sI[1].data[1]))

### Comparison method 2 &mdash; model fitting

Measure X-ray intensity by fitting a Gaussian model

In [ ]:
s.plot(navigator='slider')

In [ ]:
# Create a model based off a cropped area of signal:
m = s.isig[5.:15.].create_model()

Add background copper and cobalt elements:

In [ ]:
m.add_family_lines(['Cu_Ka', 'Co_Ka'])

Contents of the model:

In [ ]:
m.components

In [ ]:
m.plot(plot_components=True)

Fitting the model at all locations of the signal is a simple one line command:

In [ ]:
m.multifit(show_progressbar=False)

In [ ]:
m.plot(plot_components=True)

The background is fitted separately:

In [ ]:
m.fit_background()

In [ ]:
m.calibrate_energy_axis()

In [ ]:
m.plot()

Finally, we probe line intensity from the fitted model:

In [1]:
sI = m.get_lines_intensity()[:2]
sI

NameError: name 'm' is not defined

Set up the kfactors for Fe K${\alpha}$ and Pt L${\alpha}$.

In [ ]:
#From Bruker software (Esprit)
kfactors = [1.450226, 5.075602]

Quantify with Cliff Lorimer.

In [ ]:
composition = s.quantification(method="CL", intensities=sI, factors=kfactors)

In [ ]:
print('             |-----------------------------|')
print('             |     Atomic compositions     |')
print('             |-----------------------------|')

print(' \t     |  Bare core  |   BSS Signal  |')
print('|------------|-------------|---------------|')
print('| Fe (at. %) |    {:.2f}    |     {:.2f}     |'.format(composition[0].data[0], composition[0].data[1]))
print('| Pt (at. %) |    {:.2f}    |     {:.2f}     |'.format(composition[1].data[0], composition[1].data[1]))
print('|------------|-------------|---------------|')

## <a id='fur'></a> 6. Going further

<a href='#top'> Table of contents</a>

Further image processing with [scikit-image](http://scikit-image.org/) and [scipy](http://www.scipy.org/). Apply a watershed transformation to isolate the nanoparticles.

- Transform the mask into a distance map.
- Find local maxima.
- Apply the watershed to the distance map using the local maximum as seed (markers).

Adapted from this scikit-image [example](http://scikit-image.org/docs/dev/auto_examples/plot_watershed.html).

Import needed utilities from `scipy` and `skimage`:

In [ ]:
from scipy.ndimage import distance_transform_edt, label
from skimage.morphology import watershed
from skimage.feature import peak_local_max

Perform watershed segmentation:

In [ ]:
distance = distance_transform_edt(mask.data)
local_maxi = peak_local_max(distance, indices=False,
                            min_distance=2, labels=mask.data)
labels = watershed(-distance, markers=label(local_maxi)[0],
                   mask=mask.data)

Plot the results:

In [ ]:
axes = hs.plot.plot_images(
    [pt_la.T, mask.T, hs.signals.Signal2D(distance), hs.signals.Signal2D(labels)],
    axes_decor='off', per_row=2, colorbar=None, cmap='RdYlBu_r',
    label=['Pt L${\\alpha}$ intensity', 'Mask',
           'Distances', 'Separated particles'])